# Page for trying out the data

In [1]:
import pandas as pd
import time
import nltk
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN


/home/betty35/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0410 16:22:57.402264 139658517772032 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
df1 = pd.read_csv('amazon_reviews_us_Mobile_Electronics_v1_00.tsv',sep="\t", error_bad_lines=False)
# bad lines exist......

b'Skipping line 35246: expected 15 fields, saw 22\n'
b'Skipping line 87073: expected 15 fields, saw 22\n'


### DATA COLUMNS:
marketplace       - 2 letter country code of the marketplace where the review was written.  
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  
review_id         - The unique ID of the review.  
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.  
product_parent    - Random identifier that can be used to aggregate reviews for the same product.  
product_title     - Title of the product.  
product_category  - Broad product category that can be used to group reviews  
                    (also used to group the dataset into coherent parts).  
star_rating       - The 1-5 star rating of the review.  
helpful_votes     - Number of helpful votes.  
total_votes       - Number of total votes the review received.  
vine              - Review was written as part of the Vine program.  
verified_purchase - The review is on a verified purchase.  
review_headline   - The title of the review.  
review_body       - The review text.  
review_date       - The date the review was written.  

Filter out the ones not versified and get a subset of the data

In [3]:
df1 = df1.loc[df1['verified_purchase']=='Y',['review_id', 'product_id', 'product_title', 'helpful_votes','review_headline', 'review_body']]
df1['product_id'].nunique()

22299

Get a list of products that are with at least 200 reviews

In [4]:
count_df = df1.groupby('product_id').count()
count_df = count_df['review_id']

print('mean count:', count_df.mean(),'; max count:',count_df.max())

count_df = count_df.loc[lambda x: x>=200]
count_df = count_df.sort_values(ascending=False)

product_list = count_df.index.values.tolist()

mean count: 3.964034261626082 ; max count: 1044


In [5]:
df1 = df1[df1['product_id'].isin(product_list)]

df1[0:5]

,review_id,product_id,product_title,helpful_votes,review_headline,review_body
2,R2Y0MM9YE6OP3P,B00QERR5CY,iXCC Multi pack Lightning cable,0.0,great cables,These work great and fit my life proof case fo...
4,R26I2RI1GFV8QG,B0067XVNTG,Generic Car Dashboard Video Camera Vehicle Vid...,0.0,Cameras has battery issues,"Be careful with these products, I have bought ..."
48,R2WGDZBMIMZ1HK,B00LAG4HN4,"iXCC Element II Lightning Cable 6ft, iPhone Ch...",0.0,"Good, strong, and 6 feet long!","Good, strong, and 6 feet long."
60,RRPOCULNRBGQ,B00LAG4HN4,"iXCC Element II Lightning Cable 6ft, iPhone Ch...",0.0,made with excellent materials at the joints be...,Apple makes their charging products with infer...
77,R2K2WK38XR5FKZ,B00QERR5CY,iXCC Multi pack Lightning cable,0.0,One Star,Two failed


Test on one product. Get the reviews for one product

In [12]:
reviews = df1[df1['product_id']==product_list[10]]#.drop(columns=['review_id','product_title','product_id'])
product_title = reviews['product_title'].tolist()[0]
product_title

'New Trent Easypak 7000mAh Portable Triple USB Port External Battery Charger/Power Pack for Smartphones, Tablets and more (w/built-in USB cable)'

In [13]:
reviews = reviews['review_body']


reviews = reviews.str.replace('<br />','')

def remove_consecutive(text):
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text)
    return re.sub(r"([^eoEO])\1\1+",r"\1",one)

reviews = pd.Series(list(map(remove_consecutive,reviews)))

In [14]:
reviews = reviews.str.split("[.!?]+",expand=True).stack().reset_index() # split sentences

#remove empty lines
reviews['word_count'] = reviews[0].str.split().apply(len)#.strip().apply(len)
reviews = reviews.loc[reviews['word_count']>4,[0]]

reviews['text'] = reviews[0].str.strip()
reviews = reviews.drop(columns=0)

In [15]:
# Part of speech tagging
reviews['POS'] = reviews['text'].apply(nltk.word_tokenize).apply(nltk.pos_tag)

def getMainWords(pos):
    result = '';
    for x in pos:
        if x[1][0:2] in ['NN','VB']:
            result= result+x[0]+' '
    if len(result)>0:
        result= result[0:len(result)-1]
    return result

reviews['main'] = list(map(getMainWords,reviews['POS']))

In [16]:
reviews = reviews.loc[reviews['main'].apply(len)>0,['text','main']] #filtering out sentences without any nouns or verbs

reviews = reviews.reset_index(drop=True)# reset index

Word Embedding / Sentence Embedding

In [17]:
# Load google Universal Sentence Encoder
module_dir ="downloads/encoder-DNA" #"downloads/encoder"
embed = hub.Module(module_dir)

Instructions for updating:
Colocations handled automatically by placer.


W0410 16:27:45.567760 139658517772032 deprecation.py:323] From /home/betty35/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [18]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    start_time=time.time()
    reviews['sentence_embedding'] = pd.Series(list(session.run(embed(list(reviews['text'])))))
    end_time1=time.time()
    reviews['words_embedding'] = pd.Series(list(session.run(embed(list(reviews['main'])))))
    end_time2=time.time()
    print('time1:',(end_time1-start_time),' time2:',(end_time2-end_time1))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0410 16:28:04.578779 139658517772032 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0410 16:28:09.394668 139658517772032 saver.py:1483] Saver not created because there are no variables in the graph to restore


time1: 5.589915990829468  time2: 3.387993335723877


In [19]:
reviews

,text,main,sentence_embedding,words_embedding
0,Saved me a lot of times when my phone battery ...,Saved lot times phone battery needed charging ...,"[0.02959755, 0.014769347, -0.008056071, 0.0450...","[0.022527626, -0.02169827, -0.03826102, 0.0362..."
1,Size is small enough to leave in my bag to bri...,Size is leave bag bring,"[-0.01791196, 0.012329551, -0.039940733, 0.008...","[-0.05200369, 0.018965634, -0.015841357, -0.00..."
2,"If i could give it bo star , i would",i give star i,"[0.0062316796, 0.04814254, 0.01628208, -0.0619...","[-0.034200497, 0.004004612, 0.0040295785, -0.0..."
3,Don't bother signing up for new trents free pr...,Do bother signing trents product testing is wa...,"[0.011902953, 0.020841362, -0.009885815, 0.009...","[0.037173625, 0.06547706, -0.023730729, -0.041..."
4,Battery became extremely swollen breaking the ...,Battery became breaking units casing,"[0.0017779819, -0.021218482, -0.031650245, -0....","[-0.018813953, -0.041950952, -0.016895426, -0...."
5,I would careful about purchasing this unit,careful purchasing unit,"[0.01414253, -0.02439098, -0.014596826, -0.010...","[0.048511736, -0.025954328, -0.045988303, 0.02..."
6,Contacts stopped working after 1 year of use,Contacts stopped working year use,"[0.053049702, 0.053216692, -0.052431867, 0.036...","[0.03623689, 0.075679, -0.052308295, 0.0217254..."
7,went out of town did not work,went town did work,"[-0.019354776, 0.019772384, -0.022164991, -0.0...","[-0.00013442656, 0.018127868, -0.008061484, 0...."
8,very good price fast delivery works great,price delivery works,"[0.038162928, 0.006035794, -0.039759148, 0.012...","[0.0320761, 0.061937038, -0.07317903, 0.046003..."
9,I keep this in my laptop bag for my phone,keep bag phone,"[0.008420999, 0.014968383, -0.02184505, 0.0551...","[-2.63521e-06, -0.0043498627, 0.0008772601, 0...."


In [20]:
# Preparing datasets
X1 = np.array(reviews['sentence_embedding'].tolist())
X2 = np.array(reviews['words_embedding'].tolist())

In [47]:
# Clustering
#km_s = KMeans(n_clusters=10).fit(X1)  

#km_w = KMeans(n_clusters=10).fit(X2)

db_s = DBSCAN(eps=0.7).fit(X1)

db_w = DBSCAN(eps=0.7).fit(X2)

In [48]:
# Get labels
#reviews['label_km_sentence'] = km_s.labels_.tolist()
#reviews['label_km_word'] = km_w.labels_.tolist()

labels_s = db_s.labels_.tolist()
labels_w = db_w.labels_.tolist()
reviews['label_db_sentence'] = labels_s
reviews['label_db_word'] = labels_w


In [49]:
len(set(labels_s)) - (1 if -1 in labels_s else 0)

7

In [50]:
len(set(labels_w)) - (1 if -1 in labels_w else 0)

11

In [51]:
n_clusters_s = len(set(labels_s)) - (1 if -1 in labels_s else 0)
n_clusters_w = len(set(labels_w)) - (1 if -1 in labels_w else 0)


for i in range(0,n_clusters_s):
    dir1 = '/home/betty35/桌面/Capstone/workspace/data_testing/output/dbscan/'
    temp = reviews.loc[reviews['label_db_sentence']==i,['text']]
    file_name = 'sentence/'+str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')
    
for i in range(0,n_clusters_w):
    dir1 = '/home/betty35/桌面/Capstone/workspace/data_testing/output/dbscan/'
    temp = reviews.loc[reviews['label_db_word']==i,['text']]
    file_name = 'word/'+str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')

In [ ]:
# Save results for km

for i in range(0,9):
    temp = reviews.loc[reviews['label_km_sentence']==i,['text']]
    temp2 = reviews.loc[reviews['label_km_word']==i,['text']]
    file_name = 'km10/sentence/'+str(i)+'.txt'
    file_name2 = 'km10/word/'+str(i)+'.txt'
    np.savetxt(r'/home/betty35/桌面/Capstone/workspace/data_testing/output/'+file_name, temp.values, fmt='%s')
    np.savetxt(r'/home/betty35/桌面/Capstone/workspace/data_testing/output/'+file_name2, temp2.values, fmt='%s')